# Libraries

## socket_client.py

In [1]:
%%writefile socket_client.py
import socket
import time
import errno
import select
import logging

#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

sock = None

def socket_thread(p, requests):

    returnFlag = False
    cursor = 0
    while True:
        logger.debug('starting polling...')
        l = p.poll(10000)
        logger.debug('starting polling done...')
        if len(l) ==0:
            logger.debug('polling timeout, returning...')
            returnFlag = True
        #logger.debug('{}'.format(l))
        for t in l:
            #logger.debug('{}'.format(t))
            #sock = t[0]
            event = t[1]
            if(event & select.POLLERR or event & select.POLLHUP):
                logger.debug('pollerr or pollup')
                sock.close()
                returnFlag = True
                continue
            if(event & select.POLLOUT):
                try:
                    if cursor ==0:
                        logger.debug('sending data...')
                        #sock.send(requests)
                        #write(sock,requests)
                        #sock.write(requests)
                        sock.send(requests.encode())
                        
                        # We only want to send one message on this socket, in the future wait only for new incoming messages
                        p.modify(sock, select.POLLIN | select.POLLHUP | select.POLLERR)
                        logger.debug('sending data done...')
                        cursor += 1
                        continue
                    else:
                        pass
                except:
                    raise
                    # pass
            if(event & select.POLLIN):
                try:
                    logger.debug('receiving data...')
                    r = sock.recv(256)
                    logger.debug('receiving data done...')
                    # If recv() returns with 0 the other end closed the connection
                    if len(r) == 0:
                        sock.close()
                        logger.debug('len(r)=0, sock.close()')
                        returnFlag = True
                        continue
                    else:
                        # Do something with the received data...
                        logger.debug("Data received: " + str(r))
                        logger.debug("sock.close() ...")
                        sock.close()
                        logger.debug("sock.close() done ...")
                        returnFlag = True
                        return r.decode('ASCII')
                except:
                    pass
        if returnFlag is True:
            break
    return



def request(requests):
    global sock
    
    logger.debug('setting up socket...')
    # List for storing our sockets
    socket_list = []

    # Set up the first socket in non-blocking mode
    sock = socket.socket()
    sock.setblocking(True)
    sock.settimeout(10)
    # s1.setblocking(False)
    socket_list.append(sock)
    # # Set up the second socket in non-blocking mode
    # s2 = socket.socket()
    # s2.setblocking(False)
    # socket_list.append(s2)

    # Create a new poll object
    p = select.poll()
    # Register the sockets into the poll object, wait for all kind of events
    p.register(sock, select.POLLIN | select.POLLOUT | select.POLLHUP | select.POLLERR)
    # p.register(sock, select.POLLIN | select.POLLOUT | select.POLLHUP | select.POLLERR)
    logger.debug('setting up socket done...')

    for s in socket_list:
        try:
            logger.debug('connecting to the server...')
            out = s.connect(socket.getaddrinfo("192.168.1.51", 6543)[0][-1])
            logger.debug(out)
            logger.debug('connecting to the server done...')
            logger.debug('starting socket_thread...')
            response = socket_thread(p,requests)
            logger.debug('starting socket_thread done...')
            return response
        except socket.timeout:
            logger.debug('Timeout error occurred...')
        except :
            pass
    return



Overwriting socket_client.py


## kk_util.py

In [ ]:
%%writefile kk_util.py
import time
import datetime 
from ftplib import FTP
import sys
import logging
import os
from kk_util import *


def datetimerange(start_datetime, end_datetime,timedelta):
    dt = timedelta.seconds
    for i in range(int ((end_datetime - start_datetime).seconds/dt)):
        yield start_datetime + datetime.timedelta(seconds=i*dt)
        
def files_dict(IPADDRESS,ID,PW,remote_dir):
    ftp = FTP(IPADDRESS)
    ftp.login(ID,PW)
    try:
        ftp.cwd(remote_dir)
    except:
        ftp.close()
        return {}
    files_ = []
    ftp.dir(files_.append)
    ftp.close()

    files = {}
    for file in files_:
        filesize = file[25:36]
        filename = file[49:]
        files[filename] = int(filesize)
    return files


def daterange(start_date, end_date):
    for i in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(days=i)
        

def datetime_string(time_sec):
    t_ = time.localtime(time_sec)
    return '{}-{:0>2d}{:0>2d}-{:0>2d}{:0>2d}{:0>2d}'.format(t_[0],t_[1],t_[2],t_[3],t_[4],t_[5])

def date_string(time_sec):
    t_ = time.localtime(time_sec)
    return '{}-{:0>2d}{:0>2d}'.format(t_[0],t_[1],t_[2])

def getNextGridTime(T_now,T_exec):
    res = T_now%T_exec
    if res != 0:
        T_start = T_now - res + T_exec
    else:
        T_start = T_now
    return T_start

def waitUntil(T_start):
    while True:
        if time.time() >= T_start:
            break
    return

# Tasks

## task_server

In [1]:
%%writefile state0_task_svr.py
#!/usr/bin/python3

import socket_client
import datetime
import subprocess
import logging
from logging.handlers import TimedRotatingFileHandler
import time
import sys
from numpy import *
import os
import state7_upload2gcloud


os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


if 'logger' not in globals():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    # CONSOLE HANDLER
    ch = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    logger.addHandler(ch)
    # FILE HANDLER
    path = 'log/gcam-rpi-svr.log'
    fh = TimedRotatingFileHandler(path,when="midnight",backupCount=0)
    fh.setLevel(logging.DEBUG)
    formatter1 = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter1)
    logger.addHandler(fh)

def system_cmd(cmdString,cwd=None):
    if cwd==None:
        MyOut = subprocess.Popen(cmdString.split(' '),stdout=subprocess.PIPE,
                                stderr=subprocess.STDOUT)
    else:
        MyOut = subprocess.Popen(cmdString.split(' '),stdout=subprocess.PIPE,
                                stderr=subprocess.STDOUT,cwd=cwd)
    MyOut.wait()
    response = MyOut.communicate()[0]
    logger.info('STDOUT:{}'.format(response))

def system_shutdown():
    requests = 'set,rpi_state,4;'
    responses = socket_client.request(requests)
    system_cmd('/usr/bin/sudo /home/pi/wittypi/start_shutdown.sh','/home/pi/wittypi')
    
def video_rec(icam, event_timestamp):
    video_filename = 'gcam{}-{}'.format(icam,event_timestamp)
    system_cmd('/var/www/gcam/gcam/state6_video_rec.py -t 6 {}'.format(video_filename),\
               '/var/www/gcam/gcam/')
    Event(measure_time=event_timestamp).save()   
    
state_prev = 1

while True:
    requests = 'get,state;get,time;get,arms;get,icam;'
    responses = socket_client.request(requests)
    if responses:
        responses_ = responses.split(';')
        state = int(responses_[0])
        
        # DEFINE SINK CONDITIONS: ending up with a shutdown, 
        if state in [3,4,5,6,7,67]:
            acc_rms = responses_[2].split(',')[0::2]
            icam = int(responses_[3])
            event_timestamp = max(acc_rms)
            logger.info('CTLR State={}.'.format(state))
            if state in [6,67]: 
                logger.info('video recording started...')
                try:
                    video_rec(icam,event_timestamp)
                except:
                    logger.exception('exception in video_rec()') 
                logger.info('video recorded.')
            if state in [7,67]:
                logger.info('files to cloud...')
                try:
                    state7_upload2gcloud.upload()
                except:
                    logger.exception('exception in uploading to gc') 
                logger.info('files to cloud done.')
            if state in [6,7,67]:
                responses = socket_client.request('set,rpi_state,4;')
                logger.info('rpi_state=4 reported.')
            logger.info('system shutting down...'.format(state))
            # break
            system_shutdown()
        
        # DEFINE TRANSITION RULES
        if state_prev == 1:
            if state in [2,8]:
                logger.info('CTLR State={}.'.format(state))
                time_str = datetime.datetime.fromtimestamp(int(responses_[1])).strftime('%Y-%m-%d %H:%M:%S')
                system_cmd('/usr/bin/sudo /bin/date --set "{}"'.format(time_str))
                logger.info('system time updated to {}.'.format(time_str))
                logger.info('nginx starting...')
                system_cmd('/usr/bin/sudo /bin/systemctl start nginx.service')
                logger.info('nginx started.')
        elif state_prev == 2:
            if state in [8]:
                pass # do nothing
        elif state_prev == 8:
            if state in [2]:
                pass # do nothing

        # prepare next iteration
        state_prev = state
    time.sleep(10)


Overwriting state0_task_svr.py


## uploading to gcloud

In [4]:
%%writefile state7_upload2gcloud.py
#!/usr/bin/python
import datetime
from numpy import *
from kk_util import *
from ftplib import FTP
import pysftp
import sys
import logging
import os


cltr_IPADDRESS = '192.168.1.51'
cltr_incoming_dir = '/sd/data/incoming'
rpi_data_dir = 'data'
cloud_IPADDRESS = '35.189.121.145'
cloud_username = "pi"
private_key = '/home/pi/.ssh/id_rsa'
cloud_remote_dir = 'data/incoming'

if 'logger' not in globals():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler(sys.stdout)
    # formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    # ch.setFormatter(formatter)
    logger.addHandler(ch)  

#
# DOWNLOAD FILES FROM CLTLR/INCOMING TO RPI/DATA
#
# - CONNECTING
ntrial = 5
files0 = {}
    
for iter in range(ntrial):
    try:
        ftp = FTP(cltr_IPADDRESS)
        ftp.login('micro','python')
        ftp.cwd(cltr_incoming_dir)
        files0_ = []
        ftp.dir(files0_.append)
        for file in files0_:
            filesize = file[25:36]
            filename = file[49:]
            files0[filename] = int(filesize)
        break
    except:
        ftp.close()
        time.sleep(5)
        logger.debug('ftp retrial #{}...'.format(iter))
# - DOWNLOADING        
if iter == ntrial-1:
    logger.debug('ftp connection to CTRL failed after all.')
else:
    nfiles0 = len(files0)
    for idx, filename in enumerate(files0):
        local_filename = '{}/{}'.format(rpi_data_dir,filename)
        with open(local_filename,'wb') as fp:
            ftp.retrbinary('RETR '+filename,fp.write)
        if os.stat(local_filename).st_size == files0[filename]:
            ftp.delete(filename)
        logger.info('downloading {} ({}/{}) Done.'.format(filename, idx+1, nfiles0))
        if 0:
            if idx == 0: 
                break
    ftp.close()
        
#
#     UPLOADING TO CLOUD
#

files1_=os.listdir('data')  # list of files to upload
files1 = {}
for file in files1_:
    files1[file] = os.stat('data/'+file).st_size
# files1
nfiles1 = len(files1)
with pysftp.Connection(cloud_IPADDRESS, username=cloud_username, private_key=private_key) as sftp:
    with sftp.cd(cloud_remote_dir):
        for idx,file in enumerate(files1):
            response=sftp.put('data/'+file)
            if response.st_size == files1[file]:
                os.remove('data/'+file)
            logger.info('uploading {} ({}/{}) Done.'.format(file, idx+1, nfiles1))


Overwriting state7_upload2gcloud.py


In [5]:
!chmod u+x state7_upload2gcloud.py           